In [1]:
%cd /home/ubuntu/shot-glass/python

import math
from importlib import reload

import lunchbox.tools as lbt
from lunchbox.enforce import EnforceError
import pandas as pd
import numpy as np

from shot_glass.core.monad import *
from shot_glass.core.df_monad import *
from shot_glass.core.s_monad import *

/home/ubuntu/shot-glass/python
WARN (bgl): source/blender/python/generic/bgl.c:2654 BPyInit_bgl: 'bgl' imported without an OpenGL backend. Please update your add-ons to use the 'gpu' module. In Blender 4.0 'bgl' will be removed.


In [17]:
class Try(Monad):
    @classmethod
    def success(cls, value):
        # type: (Any) -> Try
        return cls(value)
    
    @classmethod
    def failure(cls, error):
        # type: (Exception) -> Try
        return cls(error)

    def __repr__(self):
        return f'{self.state.capitalize()}({self._data})'
    
    @property
    def state(self):
        if isinstance(self._data, Exception):
            return 'failure'
        return 'success'

    def fmap(self, func):
        # type: (Callable[[A], B]) -> Monad[B]
        return try_(self, func)
    
#     @classmethod
#     def _try(cls, monad, method, func):
#         data = try_(func, *args, **kwargs)
#         if isinstance(data, Exception):
#             return self.fail(data)
#         return self.wrap(data)
    
#     def bind(self, func):
#         return self._try(self, bind, func)

#     def fmap(self, func):
#         return self._try(self, fmap, func)

# Try.wrap(1).fmap(lambda x: x + 'a')
# Try.wrap(1).fmap(lambda x: x + 1)
# Try.wrap(1).wrap(Monad).fmap(lambda x: x + 'a')
# Monad.wrap(1).bind(Try.wrap).fmap(lambda x: x + 'a')
Try.wrap(Monad.wrap(1)).fmap(lambda x: x + 'a')
Monad.wrap(Try.wrap(1).fmap(lambda x: x + 'a'))

Monad(Failure(unsupported operand type(s) for +: 'int' and 'str'))

In [139]:
class Maybe(Try):
    @classmethod
    def just(cls, value):
        return cls(value)
    
    @classmethod
    def nothing(cls, value=None):
        return cls(value)
    
    @property
    def state(self):
        data = self._data
        if data is None or pd.isna(data):
            return 'nothing'
        return super().state
    
    @classmethod
    def _try(cls, monad, method, func):
        if monad.state == 'nothing':
            return cls.nothing(monad.unwrap())
        return super()._try(monad, method, func)

In [140]:
print( Maybe.wrap(None).bind(Try).__class__ )
print( Try.wrap(None).bind(Maybe).__class__ )

TypeError: Maybe._try() takes 3 positional arguments but 4 were given

In [128]:
data = pd.DataFrame()
data['foo'] = [None, 2, 3]
data['bar'] = [1, 'a', 'b']

In [129]:
m = DFMonad(data) \
    .wrap_elements(Maybe) \
    .bind_elements(Try.wrap)

# m = DFMonad(data) \
#     .wrap_elements(Try) \
#     .bind_elements(Maybe.wrap)
    
# m = DFMonad(data) \
    # .wrap_elements(Try) \
#     .bind_elements(Maybe)

m.unwrap().applymap(type)
# x = m.unwrap().loc[0, 'foo']
# x.unwrap()
# m
# m.fmap(type)
# x = m.unwrap().loc[0, 'foo']
# type(x), type(x.unwrap()), type(x.unwrap().unwrap())

,foo,bar
0,<class '__main__.Maybe'>,<class '__main__.Try'>
1,<class '__main__.Try'>,<class '__main__.Try'>
2,<class '__main__.Try'>,<class '__main__.Try'>


In [16]:
class Vertex(sgm.Monad):
    def __init__(self, data):
        super().__init__(data)
        self.validate()
    
    def validate(self):
        data = self._data
        assert len(data) == 3
        for i in data:
            assert type(i) in [int, float]

v = Vertex.wrap([2, 3, 1])
v

Vertex([2, 3, 1])